In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import re

# Load the dataset and select only the 'text' column
dataset = load_dataset('nihiluis/financial-advisor-100', split='train')

# Get the number of samples in the dataset and select all rows if it's less than 1000
num_samples = min(1000, len(dataset))
dataset = dataset.shuffle(seed=42).select(range(num_samples)).remove_columns([col for col in dataset.column_names if col != 'text'])

# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i + 1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}

# Apply the transformation to the 'text' column only
transformed_dataset = dataset.map(transform_conversation)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/539 [00:00<?, ?B/s]

(…)-00000-of-00001-f0708e72202ddcaa.parquet:   0%|          | 0.00/321k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
transformed_dataset.push_to_hub("fin-llama-100")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-987271b807de>", line 1, in <cell line: 1>
    transformed_dataset.push_to_hub("fin-llama-100")
  File "/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py", line 5418, in push_to_hub
    repo_url = api.create_repo(
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py", line 114, in _inner_fn
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py", line 3350, in create_repo
    r = get_session().post(path, headers=headers, json=json)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 637, in post
    return self.request("POST", url, data=data, json=json, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 589, in reque

TypeError: object of type 'NoneType' has no len()

In [ ]:
from datasets import load_dataset, concatenate_datasets

# Load the first dataset and select only the 'text' column
dataset1 = load_dataset('nihiluis/financial-advisor-100', split='train')

# Get the number of samples in the first dataset and select all rows if it's less than 1000
num_samples1 = min(1000, len(dataset1))
dataset1 = dataset1.shuffle(seed=42).select(range(num_samples1)).remove_columns([col for col in dataset1.column_names if col != 'text'])

# Define a function to transform the data from the first dataset
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i + 1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}

# Apply the transformation to the first dataset
transformed_dataset1 = dataset1.map(transform_conversation)

# Load the second dataset and select only the 'text' column
dataset2 = load_dataset('ihuikjkfbjk/om-financial-advisor-dataset', split='train')
dataset2 = dataset2.remove_columns([col for col in dataset2.column_names if col != 'text'])

# Combine the two datasets
combined_dataset = concatenate_datasets([transformed_dataset1, dataset2])

# Optionally, you can shuffle the combined dataset
combined_dataset = combined_dataset.shuffle(seed=42)

# Log in to Hugging Face
!huggingface-cli login

# Push the combined dataset to Hugging Face
combined_dataset.push_to_hub("fin-llama-combined")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/539 [00:00<?, ?B/s]

(…)-00000-of-00001-f0708e72202ddcaa.parquet:   0%|          | 0.00/321k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/junsume/fin-llama-combined/commit/29ebd9c589fa5f852e12a0acd5de076a7e64ebfd', commit_message='Upload dataset', commit_description='', oid='29ebd9c589fa5f852e12a0acd5de076a7e64ebfd', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

# Specify the stock ticker (e.g., Apple Inc.)
ticker = 'AAPL'
stock_data = yf.Ticker(ticker)

# Fetch news articles related to the stock
news = stock_data.news

# Print the titles and links of the news articles
for article in news:
    print(f"Title: {article['title']}")
    print(f"Link: {article['link']}\n")

Title: Warren Buffett Cuts His Stake in Apple (Again)
Link: https://finance.yahoo.com/m/ec3f41f6-0da8-3de8-92f6-1efa0c1a7f90/warren-buffett-cuts-his-stake.html

Title: This Artificial Intelligence (AI) Stock Is Down 30% From Its 52-Week Highs: Is It Worth Buying Right Now?
Link: https://finance.yahoo.com/m/51565c1a-511a-3acb-a6ed-b4d7f4cdfa7b/this-artificial-intelligence.html

Title: Inflation and retail sales data greet a roaring stock market rally: What to know this week
Link: https://finance.yahoo.com/news/inflation-and-retail-sales-data-greet-a-roaring-stock-market-rally-what-to-know-this-week-124552744.html

Title: It's Official: Apple Needs AI to Be a Winner
Link: https://finance.yahoo.com/m/1abfcc5e-7d2e-3cf4-acc1-af79d1f87ce3/it%27s-official%3A-apple-needs-ai.html

Title: Warren Buffett's Forecast Proved Incorrect -- and It's Cost Him $21 Billion Over the Last Year
Link: https://finance.yahoo.com/m/75579cb4-d2ee-3841-9a82-6f5ee71dbbf3/warren-buffett%27s-forecast.html

Title: Me

In [ ]:
!pip install requests beautifulsoup4 pandas